### Importamos Tensorflow

In [1]:
# Importaciones
import tensorflow as tf

### Verificamos la instalación de CUDA

In [2]:
!nvidia-smi

Tue May 23 03:13:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    11W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
# Verificar si CUDA está disponible y habilitar el uso de GPU
if tf.config.list_physical_devices('GPU'):
    print("CUDA está disponible. Utilizando GPU.")
    device_name = tf.test.gpu_device_name()
else:
    print("CUDA no está disponible. Utilizando CPU.")
    device_name = "/cpu:0"

CUDA está disponible. Utilizando GPU.


### Dataset

In [4]:
# Cargamos el dataser de Cifar10
cifar = tf.keras.datasets.cifar10

# Dividimos el dataset en train y test
(x_train, y_train), (x_test, y_test) = cifar.load_data()

170498071/170498071 [==============================] - 3s 0us/step


### Preprocesamiento

In [5]:
# Normalizamos los datos
model = tf.keras.applications.ResNet50(
    include_top=True,
    weights=None,
    input_shape=(32, 32, 3),
    classes=10)

### Modelo

In [6]:
# Función de loss
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# Compilamos el modelo
model.compile(optimizer="adam", loss=loss_fn, metrics=["accuracy"])

### Entrenamiento

In [7]:
import time

# Entrenamos el modelo con GPU
with tf.device(device_name):
  start_time = time.time()
  model.fit(x_train, y_train, epochs=5, batch_size=64)
  print("Tiempo de entrenamiento: %s segundos" % (time.time() - start_time))

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


782/782 [==============================] - 83s 48ms/step - loss: 1.9971 - accuracy: 0.3609
Epoch 2/5
782/782 [==============================] - 33s 42ms/step - loss: 1.7427 - accuracy: 0.4313
Epoch 3/5
782/782 [==============================] - 35s 44ms/step - loss: 1.8944 - accuracy: 0.3800
Epoch 4/5
782/782 [==============================] - 34s 43ms/step - loss: 1.6352 - accuracy: 0.4558
Epoch 5/5
782/782 [==============================] - 34s 43ms/step - loss: 1.6297 - accuracy: 0.4629
Tiempo de entrenamiento: 276.74919843673706 segundos


### Evaulación

In [8]:
# Probamos el modelo con GPU
with tf.device(device_name):
  start_time = time.time()
  model.evaluate(x_test, y_test, batch_size=64)
  print("Tiempo de evaluación: %s segundos" % (time.time() - start_time))

157/157 [==============================] - 3s 12ms/step - loss: 1.5720 - accuracy: 0.4721
Tiempo de evaluación: 3.08479380607605 segundos
